In [43]:
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV,KFold
import numpy as np
import pandas as pd

In [12]:
new_groups=datasets.fetch_20newsgroups(subset='all', 
                    categories=['alt.atheism', 'sci.space'])

In [6]:
x=new_groups.data
y=new_groups.target
tfid=TfidfVectorizer()
tfid.fit_transform(x)

<1786x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 303138 stored elements in Compressed Sparse Row format>

In [18]:
cv=KFold(shuffle=True,random_state=241)
grid={'C':np.power (10.0, np.arange(-5,6))}
clf=SVC(kernel='linear',random_state=241)
gsc=GridSearchCV(estimator=clf,param_grid=grid,cv=cv,scoring='accuracy')
gsc.fit(tfid.transform(x),y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [20]:
best_C=gsc.best_params_['C']

In [21]:
def save_answer(filename,answer):
    with open(filename+'.txt','w') as f:
        f.write(str(answer))

In [22]:
save_answer('best_C',best_C)

In [23]:
estimator=gsc.best_estimator_

In [26]:
estimator.fit(tfid.transform(x),y)

SVC(kernel='linear', random_state=241)

In [75]:
coefs=pd.DataFrame({'index':estimator.coef_.indices,
'weight':estimator.coef_.data})
coefs['weight']=coefs['weight'].apply(lambda x:abs(x))

In [76]:
coefs.sort_values(by='weight',axis=0,ascending=False,inplace=True)

In [77]:
best_idx=coefs.iloc[0:10,:]['index'].to_numpy()

In [80]:
answer=' '.join(np.sort(np.array(tfid.get_feature_names())[best_idx]))

In [82]:
save_answer('best words',answer)